NON-FUNCTIONAL

In [1]:
import pandas as pd
import numpy as np
import torch
from data_factory.DataLoader import DataLoader
from utils.utils import load_config

# models
from models.AttnAutoEncoder.AttnAutoEncoder import AutoEnc

# evaluation file
from utils.evaluation import label_evaluation

import matplotlib.pyplot as plt

import sys
sys.path.append('/media/cyprien/Data/Documents/Github/pytorch-forecasting')



DEBUG:matplotlib:matplotlib data path: /home/cyprien/anaconda3/envs/pytorch/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/cyprien/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'itertools', 'reprlib', '_collections', 'collections', '_functools', 'functools', 'contextlib', '

In [2]:
dl = DataLoader(use_previous_files=True, config_file="../config/config.yml")

INFO:data_factory.DataLoader:Use previously generated file ../data//data_export_train.csv_test.csv_fmd-False_s-True.p. Can not redo preprocessing by loading from generated file.


In [3]:
result_file = '../predict.csv'
truth_file = '../ground_truth.hdf'

## Feature Engineering

In [4]:
normal_train = dl.train[dl.train.label == 0].drop(labels='label', axis=1)
normal_val = dl.val[dl.val.label == 0].drop(labels='label', axis=1)
normal_train.shape, normal_val.shape

((2307456, 5), (115359, 5))

In [5]:
anomaly_train = dl.train[dl.train.label == 1].drop(labels='label', axis=1)
anomaly_val = dl.val[dl.val.label == 1].drop(labels='label', axis=1)
anomaly_train.shape, anomaly_val.shape

((45046, 5), (8454, 5))

In [6]:
from data_factory.preprocessing import *

normal_train_full = fill_missing_dates(normal_train, date_col='datetime', timestamp_col='timestamp', grp_col='kpi_id').copy()

In [7]:
normal_train_full['timestamp_1'] = (normal_train_full.timestamp / 60).astype(int)
max_encoder_length = 39
max_prediction_length = 1


In [8]:
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer

kpi_id_ref = normal_train_full.kpi_id.unique()[0]

X_cols = ['value_scaled', 'kpi_id', 'timestamp_1']
normal_train_full = normal_train_full.reset_index(drop=True)

is_training = normal_train_full.groupby('kpi_id').apply(
    lambda x: x.timestamp_1 < (x.timestamp_1.max() - (max_encoder_length + max_prediction_length))).reset_index(drop=True)



training = TimeSeriesDataSet(normal_train_full.loc[is_training & (normal_train_full.kpi_id == kpi_id_ref), X_cols],
                             time_idx='timestamp_1', target='value_scaled',
                             group_ids=['kpi_id'],
                             allow_missing_timesteps=False,
                             static_categoricals=['kpi_id'],
                             time_varying_unknown_reals=['value_scaled'],
                             # time_varying_known_reals=['timestamp_1'],
                             # min_encoder_length=345,
                             max_encoder_length=max_encoder_length,
                             max_prediction_length=max_prediction_length,
                             scalers={col: None for col in  ['timestamp_1', 'kpi_id']},
                             target_normalizer=None,
                             # target_normalizer=GroupNormalizer(
                             #     groups=['kpi_id'], transformation="softplus"
                             # ),
                             add_relative_time_idx=False,
                             add_target_scales=False,
                             add_encoder_length=False,
)

validation = TimeSeriesDataSet.from_dataset(training, normal_train_full[(normal_train_full.kpi_id == kpi_id_ref)], stop_randomization=True, predict=True)

In [9]:
batch_size = 128

training_dl = training.to_dataloader(train=True, batch_size = batch_size)
validation_dl = validation.to_dataloader(train=False, batch_size = batch_size)


## Model

In [89]:
from models.AttnAutoEncoder import AE

In [90]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using", device)
n_features = 1

using cuda


In [91]:
import torch
import argparse
import pandas as pd
import torch.nn as nn

from data_factory.DataLoader import DataLoader
from models.AttnAutoEncoder.train import train
from models.AttnAutoEncoder.AttnAutoEncoder import AutoEncForecast
from utils import load_config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


config_path = "../config/config.yml"
config = load_config(config_path)['AttnLSTMAutoEncoder']

df = pd.read_csv("data/AirQualityUCI.csv", index_col=config["index_col"])

train_iter = training_dl
test_iter = validation_dl

model = AutoEncForecast(config, input_size=1).to(config["device"])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

train(train_iter, test_iter, model, criterion, optimizer, config)



In [92]:
normal_train_full.value_scaled

0         -0.063041
1         -0.238350
2          0.087223
3         -0.088085
4         -0.188262
             ...   
2646266    0.472917
2646267    0.472917
2646268    0.718446
2646269    0.644206
2646270    0.489911
Name: value_scaled, Length: 2646271, dtype: float64

In [93]:
from models.AttnAutoEncoder import AutoEncForecast

index_col_name = config["index_col"] if "index_col" in config.keys() else None
df = pd.read_csv(config["data_path"], index_col=index_col_name)

if not os.path.exists(config["output_dir"]):
    os.makedirs(config["output_dir"])


model = AutoEncForecast(config, input_size=nb_features).to(config["device"])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

if config["do_eval"] and config["ckpt"]:
    model, _, loss, epoch = load_checkpoint(config["ckpt"], model, optimizer, config["device"])
    evaluate(test_iter, loss, model, config, ts)
elif config["do_train"]:
    train(train_iter, test_iter, model, criterion, optimizer, config, ts)



39

In [69]:
next(iter(training))[0]['encoder_target']

tensor([-0.0630, -0.2383,  0.0872, -0.0881, -0.1883, -0.0881, -0.0881, -0.0130,
         0.0371,  0.0121,  0.0121,  0.0121, -0.0630, -0.0130, -0.2133, -0.1632,
        -0.0380,  0.1624,  0.0371,  0.1373,  0.3627, -0.0130,  0.0121, -0.0130,
         0.0622,  0.1373,  0.5631,  0.1373,  0.0121,  0.3878,  0.0872,  0.7133,
         0.5881,  0.2124,  0.2876,  0.3377,  0.3126,  0.3377,  0.3126],
       dtype=torch.float64)

In [73]:
next(iter(training))[0]['x_cont'].squeeze(1)

tensor([-0.0630, -0.2383,  0.0872, -0.0881, -0.1883, -0.0881, -0.0881, -0.0130,
         0.0371,  0.0121,  0.0121,  0.0121, -0.0630, -0.0130, -0.2133, -0.1632,
        -0.0380,  0.1624,  0.0371,  0.1373,  0.3627, -0.0130,  0.0121, -0.0130,
         0.0622,  0.1373,  0.5631,  0.1373,  0.0121,  0.3878,  0.0872,  0.7133,
         0.5881,  0.2124,  0.2876,  0.3377,  0.3126,  0.3377,  0.3126,  0.4629],
       dtype=torch.float64)

In [74]:
next(iter(training))[1][0]

tensor([0.4629], dtype=torch.float64)

In [71]:
next(iter(training))[0].keys()

dict_keys(['x_cat', 'x_cont', 'encoder_length', 'decoder_length', 'encoder_target', 'encoder_time_idx_start', 'groups', 'target_scale'])

In [58]:
next(iter(training))[1]

(tensor([0.4629], dtype=torch.float64), None)

In [99]:
from torch import nn
from tqdm import tqdm
import copy


def train_model(model, train_dataset, val_dataset, n_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.L1Loss(reduction='sum').to(device)
    history = dict(train=[], val=[])
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1000000.0
    for epoch in range(1, n_epochs + 1):
        ## Train
        model = model.train()
        train_losses = []
        for X, (y, _) in tqdm(train_dataset):
            seq_true = torch.concat((X['encoder_cat'][:, 0].view(batch_size, 1), X['encoder_target'].view(batch_size, max_encoder_length)), dim=1).to(torch.float)
            optimizer.zero_grad()
            seq_true = seq_true.to(device)
            # print(type(seq_true), seq_true.dtype, seq_true.shape)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        val_losses = []

        ## Eval
        model = model.eval()
        with torch.no_grad():
            for X, (y, _) in val_dataset:
                seq_true = torch.concat((X['encoder_cat'][:, 0].view(batch_size, 1), X['encoder_target'].view(batch_size, max_encoder_length)), dim=1).to(torch.float)
                seq_true = seq_true.to(device)
                seq_pred = model(seq_true)
                loss = criterion(seq_pred, seq_true)
                val_losses.append(loss.item())
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [100]:
model, history = train_model(
    model,
    training_dl,
    validation_dl,
    n_epochs=150
)

100%|██████████| 978/978 [00:34<00:00, 28.36it/s]


RuntimeError: shape '[128, 1]' is invalid for input of size 1

In [20]:
def predict(model, dataset):
    predictions, losses = [], []
    criterion = nn.L1Loss(reduction='sum').to(device)
    with torch.no_grad():
        model = model.eval()
        for seq_true in dataset:
            seq_true = seq_true.to(device)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            predictions.append(seq_pred.cpu().numpy().flatten())
            losses.append(loss.item())
    return predictions, losses

Index(['timestamp', 'value', 'label', 'kpi_id', 'datetime', 'value_scaled'], dtype='object')

In [ ]:
_, losses = predict(model, train_dataset)
sns.distplot(losses, bins=50, kde=True);

In [ ]:
pred = model.predict_proba(X_train)[:, 1]
pred = np.where(pred > THRESHOLD, 1, 0)
prediction = dl.train.loc[:, ['timestamp', 'kpi_id']].rename(columns={'kpi_id': 'KPI ID'})
prediction['predict'] = pred
prediction.to_csv(result_file)

ground_truth = dl.train.loc[:, ['timestamp', 'kpi_id', 'label']].rename(columns={'kpi_id': 'KPI ID'})
ground_truth.to_hdf(truth_file, key='df')

print(label_evaluation(truth_file, result_file, delay))

In [15]:
pred = model.predict_proba(X_val)[:, 1]
pred = np.where(pred > THRESHOLD, 1, 0)
prediction = dl.val.loc[:, ['timestamp', 'kpi_id']].rename(columns={'kpi_id': 'KPI ID'})
prediction['predict'] = pred
prediction.to_csv(result_file)

ground_truth = dl.val.loc[:, ['timestamp', 'kpi_id', 'label']].rename(columns={'kpi_id': 'KPI ID'})
ground_truth.to_hdf(truth_file, key='df')

print(label_evaluation(truth_file, result_file, delay))

{"result": true, "data": 0.9149949849548645, "message": "计算成功"}


In [17]:
pred = model.predict_proba(X_test)[:, 1]
pred = np.where(pred > THRESHOLD, 1, 0)
prediction = dl.test.loc[:, ['timestamp', 'kpi_id']].rename(columns={'kpi_id': 'KPI ID'})
prediction['predict'] = pred
prediction.to_csv(result_file)